In [1]:
from ns import ns

In [2]:
networks = (1,2,3)
node_containers = []
# net_devices = {
#     "trunk": [],
#     "leaves": {x:[] for x in range(len(networks))}
# }

bridge_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]
net_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]

switch_container = ns.network.NodeContainer()
switch_container.Create(len(networks))
router_node_container = ns.network.NodeContainer()
router_node_container.Create(1)

> Note that we are creating a container for a single node. This is because of the internet stack installation several steps later.

In [3]:
for network in networks:
    node_container = ns.network.NodeContainer()
    node_container.Create(3)
    node_containers.append(node_container)

In [4]:
csma = ns.csma.CsmaHelper()

In [5]:
for idx, node_container in enumerate(node_containers):
    for node_idx in range(node_container.GetN()):
        csma_node_pair = ns.network.NodeContainer()
        csma_node_pair.Add(node_container.Get(node_idx))
        csma_node_pair.Add(switch_container.Get(idx))
        devices = csma.Install(csma_node_pair)
        bridge_device_containers[idx].Add(devices.Get(1))
        net_device_containers[idx].Add(devices.Get(0))
    csma_node_pair = ns.network.NodeContainer()
    csma_node_pair.Add(switch_container.Get(idx))
    csma_node_pair.Add(router_node_container.Get(0))
    devices = csma.Install(csma_node_pair)
    bridge_device_containers[idx].Add(devices.Get(0))
    net_device_containers[idx].Add(devices.Get(1))

## Create bridges

In [6]:
bridge_helper = ns.bridge.BridgeHelper()

In [7]:
for idx, bridge_device_container in enumerate(bridge_device_containers):
    bridge_helper.Install(switch_container.Get(idx), bridge_device_container)

### Task

Check the number of devices on switches.

In [8]:
internet = ns.internet.InternetStackHelper()

for node_container in node_containers:
    internet.Install(node_container)

# this step can cause a kernel reset if performed on nodes directly
internet.Install(router_node_container)

In [9]:
addr = ns.internet.Ipv4AddressHelper()
addr.SetBase(ns.network.Ipv4Address(f"10.0.{networks[0]}.0"),
                ns.network.Ipv4Mask("255.255.255.0"))

ifaces = []

for idx, network_id in enumerate(networks):
    device_ifaces = addr.Assign(net_device_containers[idx])
    ifaces.append(device_ifaces)
    addr.NewNetwork()

In [10]:
ifaces

In [11]:
ifaces[0].GetN()

4

## Applications

In [12]:
local_servers = ns.network.NodeContainer()
local_clients = ns.network.NodeContainer()
remote_servers = ns.network.NodeContainer()
remote_clients = ns.network.NodeContainer()

for idx, network_id in enumerate(networks):
    local_servers.Add(node_containers[idx].Get(2))
    local_clients.Add(node_containers[idx].Get(0))
    if idx == 0:
        remote_clients.Add(node_containers[idx].Get(1))
    else:
        remote_servers.Add(node_containers[idx].Get(1))

servers = ns.network.NodeContainer()
servers.Add(local_servers)
servers.Add(remote_servers)

In [13]:
ECHO_PORT = 9

In [14]:
echo_srv_helper = ns.UdpEchoServerHelper(ECHO_PORT)
srv_apps = echo_srv_helper.Install(servers)
print(f"Servers: {srv_apps.GetN()}")

Servers: 5


In [15]:
from lib.utils import get_node_ips, create_echo_client_helper

In [16]:
get_node_ips(servers.Get(0), verbose=True)

{
  "0": [
    "m_local=127.0.0.1; m_mask=255.0.0.0; m_broadcast=127.255.255.255; m_scope=0; m_secondary=0"
  ],
  "1": [
    "m_local=10.0.1.3; m_mask=255.255.255.0; m_broadcast=10.0.1.255; m_scope=2; m_secondary=0"
  ]
}


{0: [<cppyy.gbl.ns3.Ipv4InterfaceAddress object at 0x560ad2dae250>],
 1: [<cppyy.gbl.ns3.Ipv4InterfaceAddress object at 0x560ad35e2b70>]}

In [29]:
echo_clients = ns.applications.ApplicationContainer()
for local_client_idx in range(local_clients.GetN()):
    local_server_node = local_servers.Get(local_client_idx)
    # always second interface, then first address
    local_server_address = get_node_ips(local_server_node)[1][0].GetAddress().ConvertTo()
    
    echo_client_helper = create_echo_client_helper(local_server_address, ECHO_PORT)
    echo_client_app = echo_client_helper.Install(local_clients.Get(local_client_idx))
    echo_clients.Add(echo_client_app)

remote_server_address = get_node_ips(remote_servers.Get(0))[1][0].GetAddress().ConvertTo()

echo_client_helper = create_echo_client_helper(local_server_address, ECHO_PORT)
echo_client_app = echo_client_helper.Install(remote_clients.Get(0))
echo_clients.Add(echo_client_app)

remote_server_address = get_node_ips(remote_servers.Get(1))[1][0].GetAddress().ConvertTo()

echo_client_helper = create_echo_client_helper(local_server_address, ECHO_PORT)
echo_client_app = echo_client_helper.Install(remote_clients.Get(0))
echo_clients.Add(echo_client_app)

In [30]:
srv_apps.Start(ns.core.Seconds(1.0))
srv_apps.Stop(ns.core.Seconds(10.0))
echo_clients.Start(ns.core.Seconds(2.0)) # has to be later than server
echo_clients.Stop(ns.core.Seconds(10.0))

In [31]:
ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.core.LOG_LEVEL_INFO)

In [32]:
ns.Simulator.Stop(ns.Seconds(10))
ns.Simulator.Run()
ns.Simulator.Destroy()

At time +2s client sent 1500 bytes to 10.0.1.3 port 9
At time +2s client sent 1500 bytes to 10.0.3.3 port 9
At time +2s client sent 1500 bytes to 10.0.3.3 port 9
At time +2s client sent 1500 bytes to 10.0.2.3 port 9
At time +2s client sent 1500 bytes to 10.0.3.3 port 9
At time +2.00201s server received 1500 bytes from 10.0.3.1 port 49153
At time +2.00201s server sent 1500 bytes to 10.0.3.1 port 49153
At time +2.00601s server received 1500 bytes from 10.0.1.1 port 49153
At time +2.00601s server sent 1500 bytes to 10.0.1.1 port 49153
At time +2.00701s server received 1500 bytes from 10.0.2.1 port 49153
At time +2.00701s server sent 1500 bytes to 10.0.2.1 port 49153
At time +2.00802s client received 1500 bytes from 10.0.3.3 port 9
At time +2.00902s client received 1500 bytes from 10.0.2.3 port 9
At time +2.01502s client received 1500 bytes from 10.0.1.3 port 9
At time +3s client sent 1500 bytes to 10.0.1.3 port 9
At time +3s client sent 1500 bytes to 10.0.3.3 port 9
At time +3s client sen